In [2]:
cd E:\projects\ft-llm-absa-vi

E:\projects\ft-llm-absa-vi


In [3]:
from src.utils import HOTEL_CATEGORIES

In [4]:
data_dir = "data\absa-vlsp-2018"

In [5]:
dev_path = "data/absa-vlsp-2018/hotel/2-VLSP2018-SA-Hotel-dev.txt"

In [6]:

import re

In [22]:
def process_data(data_path):

    with open(data_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()

    reviews = []

    i = 0
    while i < len(lines):
        if lines[i].startswith('#'):
            id = lines[i].strip()
            review = lines[i+1].strip()
            annotation_text = lines[i+2].strip()


            # print(id, review, annotation_text)
            # print(annotation)

            # Regular expression to match the pattern
            pattern = r"\{([^#]+#[^,]+),\s*(\w+)\}"
            
            # Find all matches in the string
            matches = re.findall(pattern, annotation_text)

            # annotation = {}
            annotation = []

            for match in matches:

                category = match[0].strip()
                sentiment = match[1].strip()
                # annotation[category] = sentiment
                annotation.append((category, sentiment))

            i += 4


        reviews.append({
            "id": id,
            "review": review,
            "annotation": annotation
        })
        

    return reviews

In [24]:
test_path = "E:/projects/ft-llm-absa-vi/data/absa-vlsp-2018/hotel/3-VLSP2018-SA-Hotel-test.txt"
test_reviews = process_data(test_path)
test_reviews

[{'id': '#1',
  'review': 'Ga giường không sạch, nhân viên quên dọn phòng một ngày.',
  'annotation': [('ROOM_AMENITIES#CLEANLINESS', 'negative'),
   ('SERVICE#GENERAL', 'negative')]},
 {'id': '#2',
  'review': 'Nv nhiệt tình, phòng ở sạch sẽ, tiện nghi, vị trí khá thuận tiện cho việc di chuyển đến các địa điểm ăn + chơi Phòng có gián',
  'annotation': [('SERVICE#GENERAL', 'positive'),
   ('ROOMS#CLEANLINESS', 'neutral'),
   ('ROOMS#COMFORT', 'positive'),
   ('LOCATION#GENERAL', 'positive')]},
 {'id': '#3',
  'review': 'Đi bộ ra biển gần, tiện đi lại Phòng view biển nhưng cửa sổ view biển khá bé',
  'annotation': [('LOCATION#GENERAL', 'positive'),
   ('ROOMS#GENERAL', 'positive'),
   ('ROOMS#DESIGN&FEATURES', 'neutral')]},
 {'id': '#4',
  'review': 'Tất cả mọi thứ đều sạch sẽ, giường ngủ rất thoải mái. Không có quạt điện mà chỉ có điều hòa nên có chút bất tiện.',
  'annotation': [('HOTEL#CLEANLINESS', 'positive'),
   ('ROOM_AMENITIES#COMFORT', 'positive'),
   ('ROOM_AMENITIES#DES

# Langchain, OpenAI

In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from langchain_core.messages import AIMessage, AIMessageChunk
from langchain_core.output_parsers import StrOutputParser

In [27]:
from src.utils import HOTEL_CATEGORIES

In [28]:
# from langchain.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint
# from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_huggingface import ChatHuggingFace

In [29]:
llm = HuggingFaceEndpoint(
    # repo_id="huggingfaceh4/zephyr-7b-alpha", 
    # repo_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    # repo_id = "meta-llama/Llama-3.2-3B-Instruct",
    repo_id = "Qwen/Qwen2.5-3B-Instruct",  # 7B:15 GB mem required, exceed 10GB
    temperature=0,
    max_new_tokens=4000,
    # model_kwargs={
    #     "temperature": 0.5, 
    #     # "max_length": 64,
    #     # "max_new_tokens":512
    #               }
)


In [30]:
llm = ChatHuggingFace(llm=llm)

In [32]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0,
                 max_tokens=1000)

In [33]:
absa_prompt_template = """
You are a aspect-based sentiment analyzer in Vietnamese.
Given a text of user review, your task is to define all sentiment elements with their corresponding aspect categories, sentiment polarity.
- The "aspect category" refers to the category that aspect belongs to, and the available categories include: {categories}.
- The "sentiment polarity" refers to the degree of positivity, negativity or neutrality expressed in the opinion towards a particular aspect or feature of a product or service, and the available polarities include: "positive", "negative" and "neutral". "neutral" means mildly positive or mildly negative. Objective sentiment polarity should be ignored.

Your output should be as the following format, a list of tuples:
[(<aspect category>, <sentiment polarity>), ...]

Note, each element of the tuple has appropriate quotes around it.
Does not output any other words.

Now, start!
Input text: {text}
"""

In [ ]:
absa_prompt = PromptTemplate(template=absa_prompt_template,
                             partial_variables={"categories": HOTEL_CATEGORIES})

In [35]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [36]:
from langchain_core.output_parsers import JsonOutputParser

In [37]:
import ast

def parser(ai_message: AIMessage) -> dict:
    """
    Convert a AIMessage string representation of a list of tuples into a dictionary
    with category aspects as keys and sentiments as values.

    Parameters:
        input_data_string (str): A string representing a list of tuples.

    Returns:
        dict: A dictionary mapping category aspects to sentiments.
    """
    # Convert the string to a list of tuples
    input_data = ast.literal_eval(ai_message.content)

    # Create a dictionary to hold the output
    output_list = []

    # Populate the dictionary with category as key and sentiment as value
    for aspect in input_data:
        category = aspect[0]  # Extract the category
        sentiment = aspect[1]  # Extract the sentiment
        output_list.append((category, sentiment))

    return output_list

# # Example usage
# input_data_string = AIMessage('[("thoải mái", "FACILITIES#COMFORT", "negative", "không cảm thấy"), ("thang máy", "FACILITIES#GENERAL", "negative", "quá nóng và quá chậm chạp"), ("Bữa sáng", "FOOD&DRINKS#QUALITY", "positive", "tuyệt"), ("Phòng ngủ", "ROOMS#DESIGN&FEATURES", "positive", "tiện nghi đẹp"), ("Nhân viên", "SERVICE#GENERAL", "positive", "rất tốt")]')

# # Call the function and print the result
# output = parser(input_data_string)
# print(output)


In [38]:
absa_chain = absa_prompt | llm | parser

In [39]:
absa_chain

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={'categories': ['FACILITIES#CLEANLINESS', 'FACILITIES#COMFORT', 'FACILITIES#DESIGN&FEATURES', 'FACILITIES#GENERAL', 'FACILITIES#MISCELLANEOUS', 'FACILITIES#PRICES', 'FACILITIES#QUALITY', 'FOOD&DRINKS#MISCELLANEOUS', 'FOOD&DRINKS#PRICES', 'FOOD&DRINKS#QUALITY', 'FOOD&DRINKS#STYLE&OPTIONS', 'HOTEL#CLEANLINESS', 'HOTEL#COMFORT', 'HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL', 'HOTEL#MISCELLANEOUS', 'HOTEL#PRICES', 'HOTEL#QUALITY', 'LOCATION#GENERAL', 'ROOMS#CLEANLINESS', 'ROOMS#COMFORT', 'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 'ROOMS#MISCELLANEOUS', 'ROOMS#PRICES', 'ROOMS#QUALITY', 'ROOM_AMENITIES#CLEANLINESS', 'ROOM_AMENITIES#COMFORT', 'ROOM_AMENITIES#DESIGN&FEATURES', 'ROOM_AMENITIES#GENERAL', 'ROOM_AMENITIES#MISCELLANEOUS', 'ROOM_AMENITIES#PRICES', 'ROOM_AMENITIES#QUALITY', 'SERVICE#GENERAL']}, template='\nYou are a aspect-based sentiment analyzer in Vietnamese.\nGiven a text of user review, your task is to define a

In [40]:
test_reviews[1]

{'id': '#2',
 'review': 'Nv nhiệt tình, phòng ở sạch sẽ, tiện nghi, vị trí khá thuận tiện cho việc di chuyển đến các địa điểm ăn + chơi Phòng có gián',
 'annotation': [('SERVICE#GENERAL', 'positive'),
  ('ROOMS#CLEANLINESS', 'neutral'),
  ('ROOMS#COMFORT', 'positive'),
  ('LOCATION#GENERAL', 'positive')]}

In [41]:
input = {"text": test_reviews[99]['review']}
response = absa_chain.invoke(input)
response

[('ROOMS#DESIGN&FEATURES', 'positive'),
 ('ROOMS#COMFORT', 'positive'),
 ('FOOD&DRINKS#QUALITY', 'positive'),
 ('SERVICE#GENERAL', 'positive'),
 ('FACILITIES#GENERAL', 'positive'),
 ('FACILITIES#MISCELLANEOUS', 'neutral'),
 ('FACILITIES#MISCELLANEOUS', 'neutral')]

In [240]:

print(response)

[('HOTEL#COMFORT', 'negative'), ('HOTEL#PRICES', 'negative'), ('ROOMS#CLEANLINESS', 'neutral'), ('ROOMS#COMFORT', 'positive'), ('ROOMS#DESIGN&FEATURES', 'positive'), ('SERVICE#GENERAL', 'positive')]


---

In [42]:
from typing import List, Tuple, Dict, Union

In [43]:
import copy
import json
from tqdm.auto import tqdm

In [44]:
from src.utils import load_from_jsonl

In [53]:
def predict(model_name, 
            k_shots, 
            reviews_data: List[Dict[str, Union[str, List[Tuple[str, str]]]]],  
            n_samples: int):
    preds_list = []
    gold_list = []
    preds = []

    # predict_save_file = 

    with open(f'save/eval/hotel/pred_{model_name}_{k_shots}_shots.jsonl', 'w', encoding='utf-8') as f:
        for (i, item) in tqdm(enumerate(reviews_data[:300])):
            if i == n_samples:
                break

            input = {"text": item["review"]}
            
            gold = item["annotation"]
            gold_list.append(gold)
            
            try:
                res = absa_chain.invoke(input)
            except:
                print(f"Error at item {i+1}")
                continue
            
            preds_list.append(res)

            pred = copy.deepcopy(item)
            pred["prediction"] = res
            preds.append(pred)

            
            json_line = json.dumps(pred, ensure_ascii=False)
            f.write(json_line + '\n')
                
    return preds, preds_list, gold_list

In [54]:
len(test_reviews)

600

In [55]:
model_name = "gpt-4o-mini".lower()
k_shots = 0
n_samples = len(test_reviews)
prediction, preds_list, gold_list = predict(model_name, k_shots, test_reviews, n_samples)

0it [00:00, ?it/s]

In [63]:
preds_list[0]

[('ROOMS#CLEANLINESS', 'negative'), ('SERVICE#GENERAL', 'negative')]

In [95]:
preds_list = []
gold_list = []

for item in prediction:
    # print(item['id'])
    preds_list.append(item['prediction'])
    gold_list.append(item['annotation'])
    # break

In [71]:
# prediction

In [72]:
prediction = load_from_jsonl('E:/projects/ft-llm-absa-vi/save/eval/hotel/pred_gpt-4o-mini_0_shots_301_600.jsonl')

In [87]:
def compute_f1_scores(pred_pt, gold_pt, verbose=True):
    """
    Function to compute F1 scores with pred and gold quads
    The input needs to be already processed
    """
    # number of true postive, gold standard, predictions
    n_tp, n_gold, n_pred = 0, 0, 0

    for i in range(len(pred_pt)):
        n_gold += len(gold_pt[i])
        n_pred += len(pred_pt[i])

        for t in pred_pt[i]:
            if t in gold_pt[i]:
                n_tp += 1

    if verbose:
        print(
            f"number of gold spans: {n_gold}, predicted spans: {n_pred}, hit: {n_tp}"
        )

    precision = float(n_tp) / float(n_pred) if n_pred != 0 else 0
    recall = float(n_tp) / float(n_gold) if n_gold != 0 else 0
    f1 = 2 * precision * recall / (
        precision + recall) if precision != 0 or recall != 0 else 0
    scores = {
        'n_gold': n_gold,
        'n_pred': n_pred,
        'n_tp': n_tp,

        'precision': round(precision * 100, 2),
        'recall': round(recall * 100, 2),
        'f1': round(f1 * 100, 2)
    }

    return scores

In [96]:
pd.DataFrame(prediction)

,id,review,annotation,prediction
0,#1,"Ga giường không sạch, nhân viên quên dọn ph...","[(ROOM_AMENITIES#CLEANLINESS, negative), (SERV...","[(ROOMS#CLEANLINESS, negative), (SERVICE#GENER..."
1,#2,"Nv nhiệt tình, phòng ở sạch sẽ, tiện nghi, vị ...","[(SERVICE#GENERAL, positive), (ROOMS#CLEANLINE...","[(SERVICE#GENERAL, positive), (ROOMS#CLEANLINE..."
2,#3,"Đi bộ ra biển gần, tiện đi lại Phòng view biển...","[(LOCATION#GENERAL, positive), (ROOMS#GENERAL,...","[(LOCATION#GENERAL, positive), (ROOMS#DESIGN&F..."
3,#4,"Tất cả mọi thứ đều sạch sẽ, giường ngủ rất tho...","[(HOTEL#CLEANLINESS, positive), (ROOM_AMENITIE...","[(FACILITIES#CLEANLINESS, positive), (ROOMS#CO..."
4,#5,"Giường sạch sẽ, thoải mái, đầy đủ thiết bị cần...","[(ROOM_AMENITIES#CLEANLINESS, positive), (ROOM...","[(ROOMS#CLEANLINESS, positive), (ROOMS#COMFORT..."
...,...,...,...,...
595,#596,"vị trí thuận tiện đi lại, gần biển. bảo vệ và ...","[(LOCATION#GENERAL, positive), (SERVICE#GENERA...","[(LOCATION#GENERAL, positive), (SERVICE#GENERA..."
596,#597,Gần biển Không thích kiểu phục vụ ăn sáng.Ăn s...,"[(LOCATION#GENERAL, positive), (FOOD&DRINKS#ST...","[(LOCATION#GENERAL, positive), (SERVICE#GENERA..."
597,#598,Phục vụ chu đáo Phòng tắm k có bồn tắm,"[(SERVICE#GENERAL, positive), (ROOM_AMENITIES#...","[(SERVICE#GENERAL, positive), (ROOMS#DESIGN&FE..."
598,#599,"Phòng, nhà vệ sinh sạch sẽ. Nhân viên nhiệt tì...","[(ROOMS#CLEANLINESS, positive), (ROOM_AMENITIE...","[(ROOMS#CLEANLINESS, positive), (SERVICE#GENER..."


In [97]:
len(preds_list), len(gold_list)

(600, 600)

In [84]:
import pandas as pd
results = pd.DataFrame(columns=['id', 'review', 'annotation', 'prediction', 'n_gold', 'n_pred', 'n_hit', 'precision', 'recall', 'f1'])

In [98]:
# n_tp, n_gold, n_pred = 0, 0, 0
# pred_pt = preds_list
# gold_pt = gold_list

# for i in range(len(pred_pt)):

#     n_tp_list, n_gold_list, n_pred_list = [], [], []

#     n_gold += len(gold_pt[i])
#     n_pred += len(pred_pt[i])

#     n_gold_list.append(gold_pt[i])
#     n_pred_list.append(pred_pt[i])

#     for t in pred_pt[i]:
#         if t in gold_pt[i]:
#             n_tp += 1
#             n_tp_list.append(n_tp)



In [100]:
# results

In [101]:
scores = compute_f1_scores(preds_list, gold_list)


number of gold spans: 2584, predicted spans: 2440, hit: 1264


In [102]:
scores

{'n_gold': 2584,
 'n_pred': 2440,
 'n_tp': 1264,
 'precision': 51.8,
 'recall': 48.92,
 'f1': 50.32}